<a href="https://colab.research.google.com/github/bilalProgTech/online-data-science-ml-challenges/blob/master/MachineHack-The-Great-Indian-Hiring-Challenge/Solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget https://machinehack-be.s3.amazonaws.com/retail_price_prediction_mega_hiring_hackathon/Participants_Data_TGIH.zip
!unzip \*.zip && rm *.zip 

--2020-11-16 09:06:35--  https://machinehack-be.s3.amazonaws.com/retail_price_prediction_mega_hiring_hackathon/Participants_Data_TGIH.zip
Resolving machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)... 52.219.64.48
Connecting to machinehack-be.s3.amazonaws.com (machinehack-be.s3.amazonaws.com)|52.219.64.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6595880 (6.3M) [application/zip]
Saving to: ‘Participants_Data_TGIH.zip’

Participants_Data_T 100%[===================>]   6.29M  2.91MB/s    in 2.2s    

2020-11-16 09:06:38 (2.91 MB/s) - ‘Participants_Data_TGIH.zip’ saved [6595880/6595880]

Archive:  Participants_Data_TGIH.zip
   creating: Participants_Data_TGIH/
  inflating: Participants_Data_TGIH/Sample Submission.csv  
  inflating: __MACOSX/Participants_Data_TGIH/._Sample Submission.csv  
  inflating: Participants_Data_TGIH/Test.csv  
  inflating: __MACOSX/Participants_Data_TGIH/._Test.csv  
  inflating: Participants_Data_TGIH/Train.csv  
  

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv('/content/Participants_Data_TGIH/Train.csv')
test = pd.read_csv('/content/Participants_Data_TGIH/Test.csv')
train.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [ ]:
combine = train.append(test)
combine.shape

(406829, 8)

In [ ]:
combine['InvoiceDate'] = pd.to_datetime(combine['InvoiceDate'], format='%Y-%m-%d %H:%M:%S')
combine.dtypes

InvoiceNo               int64
StockCode               int64
Description             int64
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                 int64
dtype: object

In [ ]:
combine['Year'] = combine['InvoiceDate'].dt.year
combine['Month'] = combine['InvoiceDate'].dt.month
combine['Day'] = combine['InvoiceDate'].dt.dayofweek
combine['Hour'] = combine['InvoiceDate'].dt.hour
combine['Minute'] = combine['InvoiceDate'].dt.minute
combine['Second'] = combine['InvoiceDate'].dt.second
combine.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Year,Month,Day,Hour,Minute,Second
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35,2011,5,4,16,54,0
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35,2011,5,2,7,35,0
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35,2011,11,6,13,20,0
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35,2011,11,1,12,7,0
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35,2011,5,4,9,0,0


In [ ]:
combine = combine[['StockCode', 'Quantity', 'UnitPrice', 'Country', 'Year', 'Month', 'Day', 'Hour', 'Minute']]
combine.head()

,StockCode,Quantity,UnitPrice,Country,Year,Month,Day,Hour,Minute
0,1583,3,3.75,35,2011,5,4,16,54
1,1300,6,1.95,35,2011,5,2,7,35
2,2178,4,5.95,35,2011,11,6,13,20
3,2115,1,0.83,35,2011,11,1,12,7
4,1210,12,1.65,35,2011,5,4,9,0


In [ ]:
combine.isnull().sum()

StockCode         0
Quantity          0
UnitPrice    122049
Country           0
Year              0
Month             0
Day               0
Hour              0
Minute            0
dtype: int64

In [ ]:
def get_weekend(row):
    if row == 5 or row == 6:
        return 1
    else:
        return 0
combine['IsWeekend'] = combine['Day'].apply(get_weekend)
combine.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,StockCode,Quantity,UnitPrice,Country,Year,Month,Day,Hour,Minute,IsWeekend
0,1583,3,3.75,35,2011,5,4,16,54,0
1,1300,6,1.95,35,2011,5,2,7,35,0
2,2178,4,5.95,35,2011,11,6,13,20,1
3,2115,1,0.83,35,2011,11,1,12,7,0
4,1210,12,1.65,35,2011,5,4,9,0,0


In [ ]:
X = combine[combine['UnitPrice'].isnull()!=True].drop(['UnitPrice'], axis=1)
y = np.log1p(combine[combine['UnitPrice'].isnull()!=True]['UnitPrice'].reset_index(drop=True))

X_test = combine[combine['UnitPrice'].isnull()==True].drop(['UnitPrice'], axis=1)

X.shape, y.shape, X_test.shape

((284780, 9), (284780,), (122049, 9))

In [ ]:
X.head()

,StockCode,Quantity,Country,Year,Month,Day,Hour,Minute,IsWeekend
0,1583,3,35,2011,5,4,16,54,0
1,1300,6,35,2011,5,2,7,35,0
2,2178,4,35,2011,11,6,13,20,1
3,2115,1,35,2011,11,1,12,7,0
4,1210,12,35,2011,5,4,9,0,0


In [ ]:
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import KFold

In [ ]:
y.describe()

count    284780.000000
mean          1.161674
std           0.610241
min           0.000000
25%           0.810930
50%           1.081805
75%           1.558145
max          10.570573
Name: UnitPrice, dtype: float64

In [ ]:
err = []
y_pred_tot_lgm = []
features = X.columns
feature_importance_df = pd.DataFrame()

fold = KFold(n_splits=15)
i = 1
for train_index, test_index in fold.split(X, y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y[train_index], y[test_index]
    m = LGBMRegressor(boosting_type='gbdt',
                       max_depth=5,
                       learning_rate=0.05,
                       n_estimators=5000,
                       min_child_weight=0.01,
                       colsample_bytree=0.5,
                       random_state=1994)
    m.fit(x_train, y_train,
          eval_set=[(x_train,y_train),(x_val, y_val)],
          early_stopping_rounds=200,
          eval_metric='rmse',
          verbose=200)
    pred_y = m.predict(x_val)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = features
    fold_importance_df["importance"] = m.feature_importances_
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    print(i, "err_lgm: ",np.sqrt(mean_squared_log_error(np.exp(pred_y), np.exp(y_val))))
    err.append(np.sqrt(mean_squared_log_error(np.exp(pred_y), np.exp(y_val))))
    pred_test = m.predict(X_test)
    i = i + 1
    y_pred_tot_lgm.append(pred_test)

Training until validation scores don't improve for 200 rounds.
[200]	training's l2: 0.198313	training's rmse: 0.445324	valid_1's l2: 0.198882	valid_1's rmse: 0.445962
[400]	training's l2: 0.181342	training's rmse: 0.425843	valid_1's l2: 0.18276	valid_1's rmse: 0.427504
[600]	training's l2: 0.171432	training's rmse: 0.414043	valid_1's l2: 0.173633	valid_1's rmse: 0.416692
[800]	training's l2: 0.164283	training's rmse: 0.405319	valid_1's l2: 0.167287	valid_1's rmse: 0.409007
[1000]	training's l2: 0.159641	training's rmse: 0.399551	valid_1's l2: 0.162929	valid_1's rmse: 0.403644
[1200]	training's l2: 0.155594	training's rmse: 0.394455	valid_1's l2: 0.159231	valid_1's rmse: 0.399038
[1400]	training's l2: 0.153211	training's rmse: 0.391421	valid_1's l2: 0.157256	valid_1's rmse: 0.396555
[1600]	training's l2: 0.150723	training's rmse: 0.388231	valid_1's l2: 0.155163	valid_1's rmse: 0.393908
[1800]	training's l2: 0.148882	training's rmse: 0.385852	valid_1's l2: 0.153743	valid_1's rmse: 0.3921

In [ ]:
np.mean(err,0)

0.2980573666449592

In [ ]:
submission = pd.DataFrame()
submission['UnitPrice'] = np.exp(np.mean(y_pred_tot_lgm, 0))
submission.head()

,UnitPrice
0,3.228187
1,4.211240
2,3.197397
3,5.901172
4,4.685695


In [ ]:
submission.to_excel('submission.xlsx', index=False)